In [6]:
#Устанавливаем pySpark
# %pip install pyspark >> None
%pip install scikit-learn

  Using cached scipy-1.14.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 28.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 58.7 MB/s eta 0:00:00
Using cached scipy-1.14.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (40.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, when, max, month, year, first_value
from pyspark.sql.functions import countDistinct
from pyspark.sql import functions as F
import pandas as pd

In [2]:
# https://www.kaggle.com/datasets/dataceo/sales-and-customer-data?resource=download
df1 = pd.read_csv('sales_data2.csv', sep=",")
df2 = pd.read_csv('customer_data.csv', sep=",")
df3 = pd.read_csv('sales_data.csv', sep=",")
df = df1.merge(df2, how='left', on='customer_id')

In [7]:
from sklearn.preprocessing import LabelEncoder

df.columns = ['order_id', 'customer_id', 'product_id', 'quantity', 'price_per_unit', 'order_date', 'region', 'gender', 'age', 'payment_method']

df['total_price'] = df['quantity']*df['price_per_unit']
df['order_id'] = df['order_id'].apply(lambda x: x[1:])

df['order_id'] = df['order_id'].apply(int)
df['total_price'] = df['total_price'].apply(int)

df['customer_id'] = df['customer_id'].apply(lambda x: x[1:])

df = df[['order_id', 'product_id', 'customer_id', 'order_date', 'quantity', 'price_per_unit', 'total_price', 'payment_method', 'region']]

labelencoder = LabelEncoder()
df['product_id'] = labelencoder.fit_transform(df['product_id'])
df['order_date'] = [x[6:10]+'-'+x[3:5]+'-'+x[:2] for x in df['order_date']]

In [8]:
df.to_csv('sales_data3.csv', sep=',', index=None)
df = pd.read_csv('sales_data3.csv', sep=",")

In [9]:
df.head()

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region
0,138884,1,241288,2022-08-05,5,1500.40,7502,Credit Card,Kanyon
1,317333,4,111565,2021-12-12,3,1800.51,5401,Debit Card,Forum Istanbul
2,127801,1,266599,2021-11-09,1,300.08,300,Cash,Metrocity
3,173702,4,988172,2021-05-16,5,3000.85,15004,Credit Card,Metropol AVM
4,337046,0,189076,2021-10-24,4,60.60,242,Cash,Kanyon


In [10]:
df['order_month'] = [int(x[5:7]) for x in df['order_date']]
df.head()

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region,order_month
0,138884,1,241288,2022-08-05,5,1500.40,7502,Credit Card,Kanyon,8
1,317333,4,111565,2021-12-12,3,1800.51,5401,Debit Card,Forum Istanbul,12
2,127801,1,266599,2021-11-09,1,300.08,300,Cash,Metrocity,11
3,173702,4,988172,2021-05-16,5,3000.85,15004,Credit Card,Metropol AVM,5
4,337046,0,189076,2021-10-24,4,60.60,242,Cash,Kanyon,10


In [11]:
df.groupby(["order_month"]).agg({'total_price':['sum','mean','max']})


total_price                    
                    sum         mean    max
order_month                                
1              28886589  2488.506978  26250
2              26620674  2539.655982  26250
3              21952524  2514.607560  26250
4              18712486  2499.330306  26250
5              19716060  2561.525270  26250
6              18930564  2497.106450  26250
7              20375451  2586.701917  26250
8              19279118  2525.097315  26250
9              18792689  2555.785258  26250
10             20541837  2645.780139  26250
11             18203935  2406.972762  26250
12             19451842  2532.791927  26250

In [12]:
#Создаем SparkSession
spark=SparkSession.builder.appName('Practise').getOrCreate()
df_pyspark=spark.read.load('sales_data3.csv', format="csv", sep=",", header="true", Infer_schema=True)

# DataFrame.withColumn(colName, col). Returns a new DataFrame by adding a column or replacing the existing column that has the same name.


#необходимые преобразования
df_pyspark = df_pyspark.withColumn("order_id", df_pyspark["order_id"].cast('int'))
df_pyspark = df_pyspark.withColumn("product_id", df_pyspark["product_id"].cast('int'))
df_pyspark = df_pyspark.withColumn("customer_id", df_pyspark["customer_id"].cast('int'))
df_pyspark = df_pyspark.withColumn("order_date", df_pyspark["order_date"].cast('date'))
df_pyspark = df_pyspark.withColumn("quantity", df_pyspark["quantity"].cast('int'))
df_pyspark = df_pyspark.withColumn("price_per_unit", df_pyspark["price_per_unit"].cast('int'))
df_pyspark = df_pyspark.withColumn("total_price", df_pyspark["total_price"].cast('int'))

#добавление колонки со значением месяца
df_pyspark = df_pyspark.withColumn("order_month", month(df_pyspark["order_date"]))

#нахождение нужных значений
sales_analysis = df_pyspark.groupBy("order_month").agg(
                            sum("total_price").alias("total_sales"),
                            avg("total_price").alias("average_sales"),
                            max("total_price").alias("max_sales"))

#вывод результата с сортировкой
sales_analysis.orderBy('order_month').show()

24/07/26 15:22:04 WARN Utils: Your hostname, rusttm-ASUS-TUF resolves to a loopback address: 127.0.1.1; using 192.168.1.99 instead (on interface wlo1)
24/07/26 15:22:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/26 15:22:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/26 15:22:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+-----------+-----------+------------------+---------+
|order_month|total_sales|     average_sales|max_sales|
+-----------+-----------+------------------+---------+
|          1|   28886589|2488.5069779462438|    26250|
|          2|   26620674| 2539.655981682885|    26250|
|          3|   21952524| 2514.607560137457|    26250|
|          4|   18712486|2499.3303058634965|    26250|
|          5|   19716060| 2561.525269585553|    26250|
|          6|   18930564|2497.1064503363673|    26250|
|          7|   20375451| 2586.701916973467|    26250|
|          8|   19279118| 2525.097314996726|    26250|
|          9|   18792689|2555.7852577179383|    26250|
|         10|   20541837| 2645.780139103555|    26250|
|         11|   18203935| 2406.972762131429|    26250|
|         12|   19451842| 2532.791927083333|    26250|
+-----------+-----------+------------------+---------+



24/07/26 15:22:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


DATAFRAME.groupBy("Колонка для группировки").agg(
                                      sum("колонка для вычисления ").alias("как переименуем новую колонку"),
                                      avg("total_price").alias("average_sales"),
                                      max("total_price").alias("max_sales"))

Колонка для группировки - по каждому уникальному значанию произведем агрегацию

### Задание 2: Вычислите  (sum) количество товаров (quantity), купленных различными методами оплаты (payment_method).


Код выполняет агрегацию данных в DataFrame `df_pyspark` по столбцу `payment_method`, суммируя значения в столбце `quantity` для каждой группы и присваивая результату новое имя `num_of_sales`. В результате получается новый DataFrame `sales_analysis`, в котором каждая строка соответствует уникальному значению из столбца `payment_method`, а столбец
`num_of_sales` содержит сумму значений `quantity` для каждой группы.

- `groupBy("payment_method")` группирует данные по столбцу `payment_method`, создавая группы для каждого уникального значения в этом столбце.
- `agg(sum("quantity").alias("num_of_sales"))` применяет агрегационную функцию `sum` к столбцу `quantity` для каждой группы, суммируя значения `quantity` внутри каждой группы. Результат этой операции затем переименовывается в `num_of_sales` с помощью метода `alias`.

Таким образом, код позволяет анализировать продажи по различным методам оплаты, суммируя количество продаж для каждого метода оплаты.

In [13]:
df_pyspark.show()

+--------+----------+-----------+----------+--------+--------------+-----------+--------------+-----------------+-----------+
|order_id|product_id|customer_id|order_date|quantity|price_per_unit|total_price|payment_method|           region|order_month|
+--------+----------+-----------+----------+--------+--------------+-----------+--------------+-----------------+-----------+
|  138884|         1|     241288|2022-08-05|       5|          1500|       7502|   Credit Card|           Kanyon|          8|
|  317333|         4|     111565|2021-12-12|       3|          1800|       5401|    Debit Card|   Forum Istanbul|         12|
|  127801|         1|     266599|2021-11-09|       1|           300|        300|          Cash|        Metrocity|         11|
|  173702|         4|     988172|2021-05-16|       5|          3000|      15004|   Credit Card|     Metropol AVM|          5|
|  337046|         0|     189076|2021-10-24|       4|            60|        242|          Cash|           Kanyon|     

In [14]:
#нахождение нужных значениях
sales_analysis = df_pyspark.groupBy("payment_method").agg(sum("quantity").alias("num_of_sales"))

#вывод результата
sales_analysis.orderBy('num_of_sales').show()

+--------------+------------+
|payment_method|num_of_sales|
+--------------+------------+
|    Debit Card|       60297|
|   Credit Card|      105045|
|          Cash|      133370|
+--------------+------------+



In [15]:
df.groupby(["payment_method"]).agg({'quantity':'sum'}).sort_values('quantity')

,quantity
payment_method,
Debit Card,60297
Credit Card,105045
Cash,133370


----------------------------------------------------------

### Задание 3: Найдите регион gb(region) с самым sort_values большой sum(total_price) суммарной стоимостью продаж


1. Группируем данные в DataFrame `df_pyspark` по столбцу `region`, а затем применяет агрегационную функцию `sum` к столбцу `total_price` для каждой группы. Результат этой операции переименовывается в `total_price_per_region`.
2. Создаем новый DataFrame `sales_analysis`, содержащий сумму `total_price` для каждой уникальной `region`.
3. Выбираем столбец `region` из DataFrame `sales_analysis`, сортирует его по убыванию значений `total_price_per_region` и выбирает первую запись.
4. Выводим результат, который представляет собой регион с наибольшей суммой `total_price`.

Таким образом, код анализирует продажи по регионам, суммирует общую стоимость продаж для каждого региона и определяет регион с наибольшей суммой продаж.

In [16]:
#нахождение нужных значениях
sales_analysis = df_pyspark.groupBy("region").agg(sum("total_price").alias("total_price_per_region"))
total_price_per_region = sales_analysis.select("region").orderBy(sales_analysis.total_price_per_region.desc()).first()

#вывод результата
print(*total_price_per_region)

Mall of Istanbul


In [17]:
df.groupby(["region"]).agg({'total_price':'sum'}).sort_values('total_price', ascending=False).head(1)

,total_price
region,
Mall of Istanbul,50864070


--------------------------------------------------------------

### Задание 4: Вычислите общую сумму продаж и среднюю сумму продажи для каждого региона.


1. Группируем данные в DataFrame `df_pyspark` по столбцу `region`, а затем применяет агрегационные функции `sum` и `avg` к столбцу `total_price` для каждой группы. Результаты этих операций переименовываются в `total_price_per_region` и `avg_price_per_region` соответственно.
2. Создаем новый DataFrame `sales_analysis`, содержащий сумму `total_price` и среднее значение `total_price` для каждой уникальной `region`.
3. Выводим результаты из DataFrame `sales_analysis` с помощью метода `show()`, который отображает содержимое DataFrame в консоли.

Таким образом, код анализирует продажи по регионам, суммирует общую стоимость продаж и вычисляет среднюю стоимость продаж для каждого региона. Это позволяет получить общую картину продаж по регионам, включая как общую стоимость продаж, так и среднюю стоимость продаж в каждом регионе.

In [18]:
#нахождение нужных значениях
sales_analysis = df_pyspark.groupBy("region").agg(
    sum("total_price").alias("total_price_per_region"),
    avg("total_price").alias("avg_price_per_region")
)
#вывод результата
sales_analysis.orderBy('total_price_per_region').show()

+-----------------+----------------------+--------------------+
|           region|total_price_per_region|avg_price_per_region|
+-----------------+----------------------+--------------------+
|   Forum Istanbul|              12301819|  2486.7230644835254|
|Emaar Square Mall|              12404043|  2578.2670962377883|
|   Viaport Outlet|              12519268|   2547.673585673586|
|      Cevahir AVM|              12643034|   2533.166499699459|
|     Zorlu Center|              12898877|   2541.650640394089|
|     Istinye Park|              24614711|   2516.584296084245|
|     Metropol AVM|              25375594|  2497.3520322802874|
|        Metrocity|              37296468|   2484.609153287589|
|           Kanyon|              50545885|    2549.86051556273|
| Mall of Istanbul|              50864070|   2550.472346186632|
+-----------------+----------------------+--------------------+



In [19]:
df.groupby(["region"]).agg({'total_price':['sum','mean']}).sort_values(('total_price',  'sum'), ascending=True)

total_price             
                          sum         mean
region                                    
Forum Istanbul       12301819  2486.723064
Emaar Square Mall    12404043  2578.267096
Viaport Outlet       12519268  2547.673586
Cevahir AVM          12643034  2533.166500
Zorlu Center         12898877  2541.650640
Istinye Park         24614711  2516.584296
Metropol AVM         25375594  2497.352032
Metrocity            37296468  2484.609153
Kanyon               50545885  2549.860516
Mall of Istanbul     50864070  2550.472346

-------------------------------------------------------------------------

### Задание 5: Вычислите общее количество и сумму товаров, проданных за наличные в 2022 году.


1. Добавляем новую колонку `order_year` в DataFrame `df_pyspark`, используя функцию `year` для извлечения года из даты в колонке `order_date`.
2. Фильтруем DataFrame `df_pyspark` по двум условиям: методу оплаты должен быть "Наличные" и год заказа должен быть 2022.
3. Агрегируем данные, суммируя количество (`quantity`) и общую стоимость (`total_price`) для всех записей, удовлетворяющих условиям фильтрации. Результаты этих операций переименовываются в `total_quantity_2022` и `total_price_2022` соответственно.
4. Создаем новый DataFrame `sales_analysis`, содержащий сумму количества и общую стоимость продаж за 2022 год, оплаченных наличными.
5. Выводим результаты из DataFrame `sales_analysis` с помощью метода `show()`, который отображает содержимое DataFrame в консоли.

Таким образом, код анализирует продажи, оплаченные наличными, за 2022 год, суммируя количество и общую стоимость продаж для этого периода.

In [20]:
#добавление колонки со значением года
df_pyspark = df_pyspark.withColumn("order_year", year(df_pyspark["order_date"]))

#нахождение нужных значений
sales_analysis = df_pyspark.filter((df_pyspark.payment_method == "Cash") & (df_pyspark.order_year == 2022)).agg(
    sum("quantity").alias("total_quantity_2022"),
    sum("total_price").alias("total_price_2022")
)

#вывод результата с сортировкой
sales_analysis.show()

+-------------------+----------------+
|total_quantity_2022|total_price_2022|
+-------------------+----------------+
|              61390|        51688958|
+-------------------+----------------+



In [21]:
df['order_year'] = [int(x[:4]) for x in df['order_date']]
dfn = df[(df.payment_method=="Cash")&(df.order_year==2022)]
dfn.groupby(["payment_method"]).agg({'quantity':'sum','total_price':'sum'})

,quantity,total_price
payment_method,,
Cash,61390,51688958


### Задание 6: Найдите уникальное количество покупателей за 2022 год


1. Импортируем функцию `countDistinct` из модуля `pyspark.sql.functions`. Эта функция используется для подсчета уникальных значений в определенных столбцах DataFrame.
2. Фильтруем исходный DataFrame `df_pyspark`, оставляя только записи за 2022 год. Это делается с помощью метода `filter`, который принимает условие для фильтрации.
3. Выбираем из отфильтрованного DataFrame количество уникальных значений в столбце `customer_id` с использованием функции `countDistinct`. Результат сохраняется в новом столбце с именем `unique_users` [1].
4. Выводим результат на экран с помощью метода `show`. Этот метод отображает данные DataFrame в удобном для чтения формате.

В итоге, код подсчитывает количество уникальных клиентов (`customer_id`), совершивших покупки в 2022 году, и выводит это количество на экран.

In [22]:
#нахождение нужных значений
sales_analysis = df_pyspark.filter(df_pyspark.order_year == 2022)
sales_analysis = sales_analysis.select(countDistinct("customer_id").alias("unique_users"))

#вывод результата с сортировкой
sales_analysis.show()

+------------+
|unique_users|
+------------+
|       45551|
+------------+



In [23]:
dfn = df[(df.order_year==2022)]
dfn.groupby(["order_year"]).agg({'customer_id':'count'})

,customer_id
order_year,
2022,45551


### Задание 7: Вам даны данные с информацией о стоимости продуктов в различных валютах. Ваша задача состоит в том, чтобы перевести все цены в доллары, используя текущие курсы валют. Однако у вас есть ограничение: для некоторых продуктов курс валюты неизвестен и их стоимость должна остаться в исходной валюте. (Для конвертации из EUR в USD нужно умножить на 1.2)


1. Импортируем необходимые модули и функции из PySpark для работы с SparkSession и функцией `when` для условной логики.
2. Создаем сессию Spark с именем приложения "currency_conversion". Это необходимо для работы с Spark и выполнения операций с данными.
3. Загружаем данные в DataFrame `df` с помощью метода `createDataFrame`. Данные представляют собой список кортежей, где каждый кортеж содержит информацию о продукте: его идентификатор (`product_id`), цену (`price`) и валюту (`currency`).
4. Добавляем новый столбец `price_usd` в DataFrame, который содержит цену продукта в долларах США. Для этого используется функция `when` для условного преобразования цены в зависимости от валюты:
   - Если валюта равна "USD", цена остается без изменений.
   - Если валюта равна "EUR", цена умножается на курс обмена 1.2 (примерный курс EUR/USD).
   - Если валюта неизвестна, цена остается без изменений.
5. Выводим результат на экран с помощью метода `show`. Это позволяет увидеть, как изменились цены после конвертации в доллары США.

В итоге, код позволяет преобразовать цены продуктов из разных валют в доллары США, используя примерный курс обмена для EUR/USD.

In [24]:
# Создание сессии Spark
spark = SparkSession.builder.appName("currency_conversion").getOrCreate()

# Загрузка датасета
data = [(1, 100, "USD"),
        (2, 200, "EUR"),
        (3, 300, "Unknown"),
        (4, 100, "EUR"),
        (5, 200, "EUR"),
        (6, 300, "Unknown"),
        (7, 100, "Unknown"),
        (8, 200, "USD"),
        (9, 300, "USD")
]
df = spark.createDataFrame(data, ["product_id", "price", "currency"])

# Добавление столбца с конвертированной ценой
df = df.withColumn("price_usd", when(df.currency == "USD", df.price)
                                 .when(df.currency == "EUR", df.price * 1.2)  # Примерный курс EUR/USD = 1.2
                             	.otherwise(df.price))  # Если курс неизвестен, оставляем в исходной валюте


# df = df.withColumn("price_usd", when(df.currency == "USD", df.price).when(df.currency == "EUR", df.price * 1.2).otherwise(df.price))



# Вывод результата
df.show()

24/07/26 15:23:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+-----+--------+---------+
|product_id|price|currency|price_usd|
+----------+-----+--------+---------+
|         1|  100|     USD|    100.0|
|         2|  200|     EUR|    240.0|
|         3|  300| Unknown|    300.0|
|         4|  100|     EUR|    120.0|
|         5|  200|     EUR|    240.0|
|         6|  300| Unknown|    300.0|
|         7|  100| Unknown|    100.0|
|         8|  200|     USD|    200.0|
|         9|  300|     USD|    300.0|
+----------+-----+--------+---------+



In [25]:
dx = pd.DataFrame(data, columns = ["product_id", "price", "currency"])
dx['price_usd'] = [dx.loc[x,'price'] if dx.loc[x,'currency']=="USD" \
                      else ( dx.loc[x,'price']*1.2 if dx.loc[x,'currency']=="EUR" else  dx.loc[x,'price'] )       \
                      for x in range(len(dx))]
dx

,product_id,price,currency,price_usd
0,1,100,USD,100.0
1,2,200,EUR,240.0
2,3,300,Unknown,300.0
3,4,100,EUR,120.0
4,5,200,EUR,240.0
5,6,300,Unknown,300.0
6,7,100,Unknown,100.0
7,8,200,USD,200.0
8,9,300,USD,300.0


### Задание 8: Допустим, есть два датасета: один содержит информацию о пользователях (user_id, name, age), а другой содержит информацию о покупках пользователей (user_id, product_id, date). Необходимо найти средний возраст пользователей, совершивших покупки.


1. Инициализируем Spark сессию с именем приложения "user_purchase_join".
2. Создаем два DataFrame: `users_df` и `purchases_df`. Первый содержит информацию о пользователях (ID, имя, возраст), второй - о покупках (ID пользователя, ID продукта, дата покупки).
3. Выполняем операцию соединения (join) между этими двумя DataFrame по столбцу `user_id`. Это позволяет объединить информацию о пользователях и их покупках в одном DataFrame.
4. Группируем результат по `user_id` и вычисляет средний возраст пользователей, которые совершили покупки.
5. Выводим результат на экран.

Основная цель кода - анализировать данные о покупках пользователей, вычисляя средний возраст пользователей, совершивших покупки. Это может быть полезно для анализа поведения пользователей и оптимизации маркетинговых стратегий.

In [26]:
# Создаем Spark сессию
spark = SparkSession.builder.appName("user_purchase_join").getOrCreate()

# Загружаем датасеты
users_df = spark.createDataFrame([
	(1, "Alice", 25),
	(2, "Bob", 30),
	(3, "Charlie", 28),
	(4, "John", 56),
	(5, "Alex", 41),
	(6, "Juliya", 17)

], ["user_id", "name", "age"])

purchases_df = spark.createDataFrame([
	(1, 101, "2022-01-01"),
	(2, 102, "2022-01-02"),
	(3, 103, "2022-01-03"),
	(3, 104, "2022-01-04"),
	(6, 105, "2022-01-05")
], ["user_id", "product_id", "date"])

# Производим операцию join и вычисляем средний возраст
result_df = users_df.join(purchases_df, "user_id").groupBy("user_id").agg(avg("age").alias("average_age"))

# Выводим результат
result_df.show()

24/07/26 15:23:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------+-----------+
|user_id|average_age|
+-------+-----------+
|      1|       25.0|
|      2|       30.0|
|      3|       28.0|
|      6|       17.0|
+-------+-----------+



In [27]:
data1 = [(1, "Alice", 25),
	(2, "Bob", 30),
	(3, "Charlie", 28),
	(4, "John", 56),
	(5, "Alex", 41),
	(6, "Juliya", 17)
]
dx1 = pd.DataFrame(data1, columns = ["user_id", "name", "age"])


data2 = [	(1, 101, "2022-01-01"),
	(2, 102, "2022-01-02"),
	(3, 103, "2022-01-03"),
	(3, 104, "2022-01-04"),
	(6, 105, "2022-01-05")
]

dx2 = pd.DataFrame(data2, columns = ["user_id", "product_id", "date"])


In [28]:
dx3 = dx1.merge(dx2, how='left', on='user_id')
dx3

,user_id,name,age,product_id,date
0,1,Alice,25,101.0,2022-01-01
1,2,Bob,30,102.0,2022-01-02
2,3,Charlie,28,103.0,2022-01-03
3,3,Charlie,28,104.0,2022-01-04
4,4,John,56,NaN,NaN
5,5,Alex,41,NaN,NaN
6,6,Juliya,17,105.0,2022-01-05


In [29]:
dx4 = dx3[~dx3['product_id'].isna()].groupby(["user_id"]).agg({'age':'mean'})
dx4

,age
user_id,
1,25.0
2,30.0
3,28.0
6,17.0


### Задание 9: У вас есть два набора данных. Первый набор содержит информацию о продуктах: id продукта, название, категория и цена. Второй набор содержит информацию о заказах: id заказа, id продукта, количество. Ваша задача — использовать PySpark для выполнения следующих шагов:

1. Присоединить набор данных о продуктах к набору данных о заказах с помощью id продукта.
3. Рассчитать общую стоимость каждого заказа, учитывая количество продуктов и их цену.
4. Отфильтровать заказы, у которых общая стоимость больше 50.

1. Инициализируем сессию Spark с именем приложения "aggregate-join-filter".

2. Создаем два DataFrame (`products_df` и `orders_df`) из предопределенных списков кортежей, представляющих данные о продуктах и заказах соответственно.

3. Присоединяем DataFrame `orders_df` к DataFrame `products_df` по столбцу "product_id", используя внутреннее соединение (inner join) по умолчанию.

4. Добавляем новый столбец "total_cost" в DataFrame `joined_df`, вычисляя общую стоимость каждого заказа как произведение количества товара на цену товара.

5. Отфильтровываем заказы, у которых общая стоимость больше 50, и сохраняет результат в DataFrame `filtered_orders_df`.

6. Выводим результаты фильтрации на экран.

In [30]:
# Создание сессии Spark
spark = SparkSession.builder.appName("aggregate-join-filter").getOrCreate()

# Чтение наборов данных из CSV файлов
data_produсts = [(1, "product1", "category1", 10.0),
                (2, "product2", "category2", 15.0),
                (3, "product3", "category1", 12.5),
                (4, "product4", "category3", 20.0),
                (5, "product5", "category2", 18.0),
                (6, "product6", "category3", 25.0),
                (7, "product7", "category1", 9.0),
                (8, "product8", "category2", 16.0),
                (9, "product9", "category3", 22.0),
                (10, "product10", "category1", 11.5)]

products_df = spark.createDataFrame(data_produсts, ["product_id", "title", "category", "price"])

data_orders = [(1, 1, 5),
               (2, 3, 2),
               (3, 2, 3),
               (4, 5, 1),
               (5, 4, 4),
               (6, 7, 2),
               (7, 6, 3),
               (8, 8, 2),
               (9, 10, 1),
               (10, 9, 4)]

orders_df = spark.createDataFrame(data_orders, ["order_id", "product_id", "quantity"])

# Присоединение набора данных о продуктах к набору данных о заказах
joined_df = orders_df.join(products_df, "product_id")

# Расчет общей стоимости каждого заказа
total_cost_df = joined_df.withColumn("total_cost", F.col("quantity") * F.col("price"))

# Отфильтрование заказов с общей стоимостью больше 50
filtered_orders_df = total_cost_df.filter(total_cost_df.total_cost > 50)

# Вывод результата
filtered_orders_df.show()

24/07/26 15:23:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+--------+--------+--------+---------+-----+----------+
|product_id|order_id|quantity|   title| category|price|total_cost|
+----------+--------+--------+--------+---------+-----+----------+
|         4|       5|       4|product4|category3| 20.0|      80.0|
|         6|       7|       3|product6|category3| 25.0|      75.0|
|         9|      10|       4|product9|category3| 22.0|      88.0|
+----------+--------+--------+--------+---------+-----+----------+



In [31]:
joined_df.show()

+----------+--------+--------+---------+---------+-----+
|product_id|order_id|quantity|    title| category|price|
+----------+--------+--------+---------+---------+-----+
|         1|       1|       5| product1|category1| 10.0|
|         2|       3|       3| product2|category2| 15.0|
|         3|       2|       2| product3|category1| 12.5|
|         4|       5|       4| product4|category3| 20.0|
|         5|       4|       1| product5|category2| 18.0|
|         6|       7|       3| product6|category3| 25.0|
|         7|       6|       2| product7|category1|  9.0|
|         8|       8|       2| product8|category2| 16.0|
|         9|      10|       4| product9|category3| 22.0|
|        10|       9|       1|product10|category1| 11.5|
+----------+--------+--------+---------+---------+-----+



In [32]:
total_cost_df.show()

+----------+--------+--------+---------+---------+-----+----------+
|product_id|order_id|quantity|    title| category|price|total_cost|
+----------+--------+--------+---------+---------+-----+----------+
|         1|       1|       5| product1|category1| 10.0|      50.0|
|         2|       3|       3| product2|category2| 15.0|      45.0|
|         3|       2|       2| product3|category1| 12.5|      25.0|
|         4|       5|       4| product4|category3| 20.0|      80.0|
|         5|       4|       1| product5|category2| 18.0|      18.0|
|         6|       7|       3| product6|category3| 25.0|      75.0|
|         7|       6|       2| product7|category1|  9.0|      18.0|
|         8|       8|       2| product8|category2| 16.0|      32.0|
|         9|      10|       4| product9|category3| 22.0|      88.0|
|        10|       9|       1|product10|category1| 11.5|      11.5|
+----------+--------+--------+---------+---------+-----+----------+



### Задание 10: Найти сумму чисел в колонке


1. Создаем сессию Spark с именем приложения "sum_example".
2. Создаем DataFrame с одним столбцом "number", содержащим числа от 1 до 4.
3. Вычисляем сумму всех чисел в столбце "number" и выводит результат.

Код использует функцию `sum` из модуля `pyspark.sql.functions`, которая является агрегатной функцией и возвращает сумму всех значений в указанном столбце. В данном случае, она применяется к столбцу "number" DataFrame `df` для вычисления общей суммы чисел в этом столбце. Результат вычисления суммы затем отображается с помощью метода `show()`.

In [33]:
# Создание Spark сессии
spark = SparkSession.builder.appName("sum_example").getOrCreate()

# Создание DataFrame
data = [(1,), (2,), (3,), (4,)]
df = spark.createDataFrame(data, ["number"])

# Найти сумму чисел в колонке "number"
sum_result = df.select(sum("number")).show()

+-----------+
|sum(number)|
+-----------+
|         10|
+-----------+



24/07/26 15:24:02 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Задание 11: Посчитать количество уникальных значений в колонке


1. Импортируем необходимые модули из библиотеки PySpark для работы с Spark и функцией `countDistinct` для подсчета уникальных значений в DataFrame.
2. Создаем экземпляр SparkSession с именем приложения "count_distinct_example". Это необходимо для инициализации Spark и подготовки среды для выполнения операций с данными.
3. Создаем DataFrame с именем `df`, используя предоставленные данные. В данном случае, DataFrame содержит одну колонку "name" с именами "Alice", "Bob", "Alice" и "Eve".
4. Выполняем подсчет количества уникальных значений в колонке "name" с помощью функции `countDistinct` и выводит результат на экран с помощью метода `show()`.

В результате выполнения кода будет выведено количество уникальных имен в колонке "name", что в данном случае равно 3, так как имена "Alice" и "Bob" встречаются дважды, а "Eve" - один раз.

In [34]:
# Создание Spark сессии
spark = SparkSession.builder.appName("count_distinct_example").getOrCreate()

# Создание DataFrame
data = [("Alice",), ("Bob",), ("Alice",), ("Eve",)]
df = spark.createDataFrame(data, ["name"])

# Посчитать количество уникальных значений в колонке "name"
count_result = df.select(countDistinct("name")).show()

24/07/26 15:24:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+
|count(DISTINCT name)|
+--------------------+
|                   3|
+--------------------+



### Задание 12: Выполнить фильтрацию данных по определенному условию


Данный код выполняет следующие действия:

1. Импортируем класс `SparkSession` из модуля `pyspark.sql`, который необходим для создания сессии Spark.
2. Создаем сессию Spark с именем приложения "filter_example".
3. Создаем список кортежей `data`, содержащий пары имя-возраст.
4. Создаем DataFrame `df` из списка `data`, указывая имена столбцов как "name" и "age".
5. Фильтруем данные в DataFrame `df`, оставляя только те строки, где возраст меньше 30, и выводит результат на экран с помощью метода `show()`.

Важно отметить, что метод `filter()` используется для фильтрации строк в DataFrame на основе заданного условия. В данном случае, условием является `df.age < 30`, что означает выборку строк, где значение в столбце "age" меньше 30. Результатом выполнения этого кода будет отображение на экране строк DataFrame, соответствующих этому условию.

In [35]:
 # Создание Spark сессии
spark = SparkSession.builder.appName("filter_example").getOrCreate()

# Создание DataFrame
data = [("Alice", 25), ("Bob", 30), ("Eve", 20), ("Charlie", 35)]
df = spark.createDataFrame(data, ["name", "age"])

# Выполнить фильтрацию данных по возрасту младше 30
filtered_data = df.filter(df.age < 30).show()

24/07/26 15:24:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----+---+
| name|age|
+-----+---+
|Alice| 25|
|  Eve| 20|
+-----+---+



### Домашнее задание

### Условие: дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

Данные для таблицы:


In [ ]:
data = {'id': [1, 2, 3, 4], 'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary':['70', '80', '60', '90'], 'ManagerId':['3', '4', 'Null', 'Null']}
df = pd.DataFrame(data)
df

,id,Name,Salary,ManagerId
0,1,Joe,70,3
1,2,Henry,80,4
2,3,Sam,60,Null
3,4,Max,90,Null


Результат должен быть:
Joe (табличка с одной строкой и одним столбцом со значением Joe)


In [ ]:
# Решение

In [ ]:
# Импортируем модуль Apache Spark - Spark SQL
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Создаем (инициализируем) SparkSession
spark = SparkSession.builder.appName("EmployeeSalaryComparison").getOrCreate()

#  Составляем data - лист из кортежей элементов таблицы
data = [(1, "Joe", 70, 3),
        (2, "Henry", 80, 4),
        (3, "Sam", 60, None),
        (4, "Max", 90, None)]

# Создаем DataFrame
df = spark.createDataFrame(data, ["Id", "name", "Salary", "managerId"])

# Выводим созданную таблицу
df.show()

+---+-----+------+---------+
| Id| name|Salary|managerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     NULL|
|  4|  Max|    90|     NULL|
+---+-----+------+---------+



In [ ]:
# Присоединение DataFrame к самому себе для сопоставления сотрудников с их менеджерами
joined_df = df.alias("emp").join(df.alias("mgr"), col("emp.managerId") == col("mgr.id"))

joined_df.show()

+---+-----+------+---------+---+----+------+---------+
| Id| name|Salary|managerId| Id|name|Salary|managerId|
+---+-----+------+---------+---+----+------+---------+
|  1|  Joe|    70|        3|  3| Sam|    60|     NULL|
|  2|Henry|    80|        4|  4| Max|    90|     NULL|
+---+-----+------+---------+---+----+------+---------+



In [ ]:
# Фильтрация данных, чтобы выбрать сотрудников, которые зарабатывают больше своих менеджеров
result_df = joined_df.filter(col("emp.salary") > col("mgr.salary")).select(col("emp.name"))

result_df.show()

+----+
|name|
+----+
| Joe|
+----+

